In [1]:
## import libraries
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from torch import nn
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from pathlib import Path
import json
import pickle
import os
import sys

In [2]:
#set device for pytorch
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
#same class as from training
D = 5
K = 1
nodes = 2 ** 8

class carlearner(nn.Module):
    def __init__(self):
        super().__init__()
        self.input = nn.Sequential(nn.Linear(D, nodes), 
                                   nn.ReLU())
        self.mid1 = nn.Sequential(nn.Linear(nodes, nodes), 
                                 nn.ReLU(),
                                nn.Linear(nodes, nodes), 
                                 nn.ReLU())
        self.mid2 = nn.Sequential(nn.Linear(nodes, nodes), 
                                  nn.ReLU(),
                                nn.Linear(nodes, nodes), 
                                 nn.ReLU())
        self.output = nn.Linear(nodes, K)
    
    def forward(self, x):
        
        return self.output(self.mid2(self.mid1(self.input(x)))).squeeze(-1)

In [4]:
#instance of class
model = carlearner().to(device)

In [5]:
#load model parameters
if os.path.exists('label_normal.pth'):
        print("Loading model")
        model.load_state_dict(torch.load('label_normal.pth'))
else:
    print("No model, please run training notebook.")

Loading model


In [6]:
with open('model_key.json', 'r') as file:
    model_key_dict = json.load(file)
with open('transmission_key.json', 'r') as file:
    transmission_key_dict = json.load(file)
with open('fuel_key.json', 'r') as file:
    fuel_key_dict = json.load(file)

In [7]:
model_key = pd.DataFrame(list(model_key_dict.items()), columns=['encoded_model', 'model'])

transmission_key = pd.DataFrame(list(transmission_key_dict.items()), columns=['encoded_transmission', 'transmission_type'])

fuel_key = pd.DataFrame(list(fuel_key_dict.items()), columns=['encoded_fuel', 'fuel_type'])

In [8]:
with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

In [9]:
def price(car,
         trans,
         fuel,
         year,
         mile):
    em = model_key.loc[model_key['model'] == car]
    em = em.index[0]
    
    et = transmission_key.loc[transmission_key['transmission_type'] == trans]
    et = et.index[0]
    
    ef = fuel_key.loc[fuel_key['fuel_type'] == fuel]
    ef = ef.index[0]
    YM = scaler.transform(np.array((year, mile)).reshape(1, -1))
    year = YM[0, 0]
    #print(year)
    mile = YM[0, 1]
    test = torch.tensor([year, mile, em, et, ef])
    return model(test.to(device).type(torch.float32)) * 0.88

def plotyear(car: str,
               trans: str,
               fuel: str,
               mile,
               year1 = 2005,
               year2 = 2023):
    em = model_key.loc[model_key['model'] == car]
    em = em.index[0]
    
    et = transmission_key.loc[transmission_key['transmission_type'] == trans]
    et = et.index[0]
    
    ef = fuel_key.loc[fuel_key['fuel_type'] == fuel]
    ef = ef.index[0]
    
    array = np.zeros(year2 - year1 + 1)
    
    for i in range(year1, year2 + 1):
        YM = scaler.transform(np.array((i, mile)).reshape(1,-1))
        i1 = YM[0, 0]
        mile = YM[0, 1]
        test = torch.tensor([i1, mile, em, et, ef])
        price = model(test.to(device).type(torch.float32)) * 0.88
        #print(i, price)
        array[i - year1] = price
    
    x = np.linspace(year1, year2, year2 - year1 + 1)
    plt.scatter(x, array)

In [10]:
carmodel = str(input("Please enter a car model: ")).title()
try:
    model_key.loc[model_key['model'] == carmodel].index[0]
except:
    print("We couldn't find that car.")
    sys.exit(1)


cartrans = str(input("Please enter the transmission type: ")).title()
try:
    transmission_key.loc[transmission_key['transmission_type'] == cartrans].index[0]
except:
    print("We couldn't find that.")
    sys.exit(1)


carfuel = str(input("Please enter the fuel type: ")).title()
try:
    fuel_key.loc[fuel_key['fuel_type'] == carfuel].index[0]
except:
    print("We couldn't find that.")
    sys.exit(1)


caryear = int(input("Please enter the registration year: "))


carmile = int(input("Please enter the mileage: "))

price(carmodel, cartrans, carfuel, caryear, carmile)

Please enter a car model: Dacia Sandero
Please enter the transmission type: Manual
Please enter the fuel type: Petrol
Please enter the registration year: 2019
Please enter the mileage: 120000


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


tensor(13369.4648, device='cuda:0', grad_fn=<MulBackward0>)